Predict whether posts come from depression or suicide thread

In [22]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
import numpy as np

In [28]:
df_suicide_posts = pd.read_csv('suicidewatch_posts.csv')
df_depression_posts = pd.read_csv('depression_posts.csv')

In [29]:
# Create column 'Suicide', intialize with 1
df_suicide_posts['Suicide'] = 1
df_suicide_posts['Depression'] = np.nan

print(len(df_suicide_posts)) # Check
df_suicide_posts.head(1) # Check

4447


,title,score,p_id,subreddit,url,num_comments,body,p_timestamp,c_id,comment,c_timestamp,Post_Reply,Time_to_Comment,Suicide,Depression
0,New wiki on how to avoid accidentally encourag...,596,cz6nfd,SuicideWatch,https://www.reddit.com/r/SuicideWatch/comments...,157,We've been seeing a worrying increase in pro-s...,2019-09-03 18:49:51,eyx1qcc,I'm sorry did you say suicide fetishists? As i...,2019-09-03 23:34:05,Y,0 days 04:44:14.000000000,1,NaN


In [30]:
# Create column 'Depression', intialize with 0
df_depression_posts['Suicide'] = np.nan
df_depression_posts['Depression'] = 0

print(len(df_depression_posts)) # Check
df_depression_posts.head(1) # Check

4035


,title,score,p_id,subreddit,url,num_comments,body,p_timestamp,c_id,comment,c_timestamp,Post_Reply,Time_to_Comment,Suicide,Depression
0,Regular Check-In Post,970,cml6ni,depression,https://www.reddit.com/r/depression/comments/c...,5523,Welcome to /r/depression's check-in post - a p...,2019-08-06 06:12:33,ew4joli,I gave in to my depression and stayed home ins...,2019-08-06 20:21:04,Y,0 days 14:08:31.000000000,NaN,0


In [31]:
# Merge the two datasets
frames = [df_depression_posts, df_suicide_posts]
df_merged = pd.concat(frames)

print(len(df_merged)) # Check
df_merged.head(1) # Check

8482


,title,score,p_id,subreddit,url,num_comments,body,p_timestamp,c_id,comment,c_timestamp,Post_Reply,Time_to_Comment,Suicide,Depression
0,Regular Check-In Post,970,cml6ni,depression,https://www.reddit.com/r/depression/comments/c...,5523,Welcome to /r/depression's check-in post - a p...,2019-08-06 06:12:33,ew4joli,I gave in to my depression and stayed home ins...,2019-08-06 20:21:04,Y,0 days 14:08:31.000000000,NaN,0.0


In [34]:
# Split into train and test set
df_merged['Target'] = df_merged['Suicide'].combine_first(df_merged['Depression'])

df_merged['Target'].value_counts() # Check
df_merged.head(1)

,title,score,p_id,subreddit,url,num_comments,body,p_timestamp,c_id,comment,c_timestamp,Post_Reply,Time_to_Comment,Suicide,Depression,Target
0,Regular Check-In Post,970,cml6ni,depression,https://www.reddit.com/r/depression/comments/c...,5523,Welcome to /r/depression's check-in post - a p...,2019-08-06 06:12:33,ew4joli,I gave in to my depression and stayed home ins...,2019-08-06 20:21:04,Y,0 days 14:08:31.000000000,NaN,0.0,0.0


In [ ]:
# Train and run Logistic Regression model
# Create design matrices: see Deepayan Classification 3 Logistic Regression notebook

# MSE accuracy